<a href="https://colab.research.google.com/github/miketriana/cap4630-wocjan/blob/master/HW_4/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Homework 4

In this assignment, I will be implementing functions related to convolutional neural networks.

##Problem 1

In this problem I will use numpy to implement the function `conv2d`. It will take in an input matrix `input_mat` and a kernel / filter matrix `kernel_mat` and compute the convolution of the two matrices, returning the result `output_mat`.

In [0]:
import numpy as np

In [0]:
# Algorithm of order O(n^2 * m^2), where n is width of input matrix and m is width of kernel

def conv2d(input_mat, kernel_mat):
    # Kernel dimensions define dimensions of tiles - mxm
    inp_width = input_mat.shape[0]
    if (inp_width != input_mat.shape[1]):
        raise ValueError("Cannot compute, input matrix is not a square!")

    k_width = kernel_mat.shape[0]
    if (k_width != kernel_mat.shape[1]):
        raise ValueError("Cannot compute, kernel matrix is not a square!")

    if (k_width > inp_width):
        raise ValueError("Cannot compute, kernel size is larger than input matrix!")

    output_mat = np.zeros((inp_width - k_width + 1, inp_width - k_width + 1))

    # Slide across each mxm tile
    for row in range(inp_width - k_width + 1):
        for col in range(inp_width - k_width + 1):
            tile = input_mat[row:row + k_width, col:col + k_width]

            # Apply filter to tile, aka compute the convolution
            sum = 0
            for i in range(k_width):
                for j in range(k_width):
                    sum += input_mat[row + i][col + j] * kernel_mat[-i - 1][-j - 1]

            # Store result in output matrix
            output_mat[row][col] = sum

    return output_mat

###Test Cases

Adapted from https://colab.research.google.com/drive/1MIoNq4_xglLesBF1kdWJelP2CmdPuYbv?authuser=1

In [3]:
from scipy import signal
import numpy as np

input_mat = []
kernel_mat = []
expected_mat = []

# test case 1
input_mat.append(np.array([[1, 2, 1, 2],
                      [2, 1, 2, 1],
                      [1, 2, 1, 2],
                      [2, 1, 2, 1]]))

kernel_mat.append(np.array([[1, 0],
                       [0, 1]]))

expected_mat.append(np.array([[2, 4, 2],
                                [4, 2, 4],
                                [2, 4, 2]]))

# test case 2
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, 0], [0, 1]]))
expected_mat.append(np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]))


# test case 3
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, -1],
                       [-1, 0]]))

expected_mat.append(np.array([[ 1, -1,  0], [-1,  1, -1],[ 0, -1,  1]]))


# test case 4
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))

expected_mat.append(np.array([[4]]))


# test case 5 - should either through an error, or return empty matrix
input_mat.append(np.array([[1, -1],
                       [-1, 0]]))
kernel_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))

expected_mat.append([])



for i in range(len(input_mat)):
  # uncomment line for student code testing
  try:
    output_mat = conv2d(input_mat[i], kernel_mat[i])
  except Exception as error:
    print(repr(error))
    output_mat = []

  # uncomment lines below (and comment line above) for generating test cases.
  #if input_mat[i].shape[0] < kernel_mat[i].shape[0]:
  #  output_mat = []
  #else:
  #  output_mat = signal.convolve2d(input_mat[i], kernel_mat[i], mode='valid')

  print(output_mat)
  if np.array_equal(output_mat, expected_mat[i]):
    print("Correct output!\n")
  else:
    print("Incorrect output!\n")

[[2. 4. 2.]
 [4. 2. 4.]
 [2. 4. 2.]]
Correct output!

[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]]
Correct output!

[[ 1. -1.  0.]
 [-1.  1. -1.]
 [ 0. -1.  1.]]
Correct output!

[[4.]]
Correct output!

ValueError('Cannot compute, kernel size is larger than input matrix!',)
[]
Correct output!



##Problem 2

In this problem I will use numpy to implement the function `maxpooling2d`. It will take in an input matrix `input_mat` and an integer `s` and compute a maxpooling operation on the matrix using an sxs window, returning the result `output_mat`.

In [0]:
# Algorithm of order O(n^2 * s^2), where n is width of input matrix and s is window size

def maxpooling2d(input_mat, s):
    inp_width = input_mat.shape[0]
    if (inp_width != input_mat.shape[1]):
        raise ValueError("Cannot compute, input matrix is not a square!")

    if (s > inp_width):
        raise ValueError("Cannot conpute, window size is larger than input matrix!")

    output_mat = np.zeros((int(inp_width / s), int(inp_width / s)))

    # Analyze each sxs tile
    for row in range(0, inp_width - s + 1, s):
        for col in range(0, inp_width - s + 1, s):
            tile = input_mat[row:row + s, col:col + s]

            # Find max value within tile
            m = float("-inf")
            for i in range(s):
                for j in range(s):
                    m = max(m, input_mat[row + i][col + j])

            # Store result in output matrix
            output_mat[int(row / s)][int(col / s)] = m

    return output_mat

###Test Cases

Adapted from https://colab.research.google.com/drive/1MIoNq4_xglLesBF1kdWJelP2CmdPuYbv?authuser=1

In [5]:
import skimage.measure

input_mat = []
expected_mat = []
s = []
input_mat.append(np.array([[1, 2, 1, 2],
                      [2, 4, 2, 1],
                      [1, 2, 4, 2],
                      [2, 1, 2, 1]]))
s.append(2)

expected_mat.append(np.array([[4, 2],
                                [2, 4]]))

input_mat.append(np.array([[1, 2, 1, 2, 4, 5],
                      [2, 4, 2, 1, 0, 3],
                      [1, 2, 4, 2, 4, 5],
                      [2, 1, 2, 1, 2, 1],
                      [1, 1, 2, 3, 1, 2],
                      [1, 1, 2, 3, 1, 2]]))
s.append(2)

expected_mat.append([[4, 2, 5],
 [2, 4, 5],
 [1, 3, 2]])

input_mat.append(np.array([[1, 2, 1, 2, 4],
                      [2, 4, 2, 1, 0],
                      [1, 2, 4, 2, 4],
                      [2, 1, 2, 1, 2],
                      [1, 1, 2, 3, 1]]))
s.append(2)

expected_mat.append([[4, 2],
 [2, 4]])


for i in range(len(input_mat)):
  # uncomment top line and comment second line to test code
  try:
    output_mat = maxpooling2d(input_mat[i], s[i])
  except Exception as error:
    print(repr(error))
    output_mat = []
  #output_mat = skimage.measure.block_reduce(input_mat[i], (s[i],s[i]), np.max)

  print(output_mat)

  if np.array_equal(output_mat, expected_mat[i]):
    print("Correct output!")
  else:
    print("Incorrect output!")


[[4. 2.]
 [2. 4.]]
Correct output!
[[4. 2. 5.]
 [2. 4. 5.]
 [1. 3. 2.]]
Correct output!
[[4. 2.]
 [2. 4.]]
Correct output!
